In [1]:
import pandas as pd

In [27]:
import numpy as np
from sklearn import decomposition
from scipy import linalg
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import string
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings('ignore')

c:\users\abhi\anaconda3\envs\newsscraper\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [5]:
## normalize and clean summaries
import re
# # lower case
# for i in range(len(train_x)):
#     train_x[i] = train_x[i].lower()
#remove \r and \n, lowercase, and only keep alhpa characters
def clean_data(text):
    for i in range(len(text)):
        #train_x[i] = re.sub(r'[\s\\@&#?!\x92]',' ',train_x[i])
        #train_x[i] = re.sub(r'[^a-zA-Z0-9]',' ',train_x[i])
        text[i] = text[i].lower()
        text[i] = text[i].translate(str.maketrans('','',string.punctuation)) #remove punctuation
        text[i] = re.sub(r'[^a-zA-Z]',' ',text[i]) #only keep alpha characters
        
    return text

In [12]:
# lemmatize words, remove stop words, and keep word with >3 letters
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self,doc):
        return [self.wnl.lemmatize(t,pos='v') for t in word_tokenize(doc) if t not in stopwords.words('english') and len(t)>3]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
# Get all individual news dataframes

breitbart_df = pd.read_csv('breitbart_articles.csv')
#breitbart_df.head()

foxnews_df = pd.read_csv('FOXNews_articles.csv')
#foxnews_df.head()

nbc_titles = pd.read_csv('nbc_articles.csv')
nbc_summ = pd.read_csv('nbc_summaries.csv')
nbc_df = pd.merge(nbc_titles,nbc_summ,how='inner',on='article_link')
#nbc_df.head()

nytimes_df = pd.read_csv('NYTimes_articles.csv',encoding='latin')
#nytimes_df.head()

allsides_1 = pd.read_csv('News_278.csv')
allsides_2 = pd.read_csv('News_370.csv')
allsides_df = allsides_1.append(allsides_2,ignore_index=True,sort=False)
allsides_df.head()

,news_title,news_summary,news_source,news_bias,news_lin
0,Democrats will meet in the next few weeks to d...,House Democrats will hold a meeting to discuss...,Washington Post,Political News Media Bias Rating: Lean Left,https://www.allsides.com/news/2019-04-21-0834/...
1,Why It Matters That Trump and Michael Cohen Ha...,"Fixer. The title wasn’t a formal one, but Mich...",New York Times - News,Political News Media Bias Rating: Lean Left,https://www.allsides.com/news/2019-04-21-0832/...
2,Confused about who’s who in the Mueller report...,"The Mueller report will be released Thursday, ...",Vox,Political News Media Bias Rating: Left,https://www.allsides.com/news/2019-04-18-0720/...
3,Five key things to look for in the Mueller report,Barr has said the report has two parts: one on...,The Guardian,Political News Media Bias Rating: Lean Left,https://www.allsides.com/news/2019-04-18-0708/...
4,George Conway Reveals The Key Phrase To Watch ...,Attorney General William Barr has said he will...,HuffPost,Political News Media Bias Rating: Left,https://www.allsides.com/news/2019-04-15-0925/...


In [49]:
breitbart = breitbart_df.drop(['image','img_caption'],axis=1)
breitbart.head()

,title,summary,source,article_link
0,Adam Schiff: America Will Not Survive Another ...,"Sunday on ABC’s “This Week,” House Intelligenc...",Breitbart,/clips/2019/05/12/adam-schiff-america-will-not...
1,"Schiff: We Are Considering $25,000 a Day Fines...",NaN,Breitbart,/clips/2019/05/12/schiff-we-are-considering-25...
2,Delingpole: Viva Brexit! Nigel Farage Totally ...,NaN,Breitbart,/europe/2019/05/12/farage-destroys-marr-shows-...
3,Kamala Harris: If Elected I Will Take Gun Deal...,NaN,Breitbart,/clips/2019/05/12/kamala-harris-if-elected-i-w...
4,China Strikes Back: Tariffs on U.S. Goods to R...,The trade dispute escalated Monday when China ...,Breitbart,/economy/2019/05/13/china-strikes-back-tariffs...


In [86]:
foxnews = foxnews_df.drop(['topic'],axis=1)
foxnews = foxnews.rename(index=str,columns={'link':'article_link'})
foxnews.head()

,title,summary,source,article_link
0,How House Republicans are making life difficul...,"House Republicans are driving Democrats crazy,...",FOXNews,/web/20190301092529/https://www.foxnews.com/po...
1,Conservative critics accuse Ilhan Omar of new ...,"Minnesota Democratic Rep. Ilhan Omar, whose pr...",FOXNews,/web/20190301092529/https://www.foxnews.com/po...
2,"Cory Booker, other 2020 Dem presidential hopef...",New Jersey Democratic Sen. Cory Booker introdu...,FOXNews,/web/20190301092529/https://www.foxnews.com/po...
3,Ocasio-Cortez warns of 'list' for moderate Dem...,After more than two dozen moderate Democrats b...,FOXNews,/web/20190301092529/https://www.foxnews.com/po...
4,"Despite Trump-Kim summit collapse, US-North Ko...",Denuclearization talks between President Trump...,FOXNews,/web/20190301092529/https://www.foxnews.com/po...


In [80]:
nbc = nbc_df[['title','summary','source','article_link']]
nbc.head()

,title,summary,source,article_link
0,Most Americans don't think Trump is in the cle...,Most Americans don't think Trump is in the cle...,NBC News,https://web.archive.org/web/20190401012911/htt...
1,Beto O'Rourke kicks off campaign in Texas with...,Beto O'Rourke kicks off three Texas rallies wi...,NBC News,https://web.archive.org/web/20190401012911/htt...
2,Growing student debt crisis: Candidates say ca...,Growing student debt crisis: Candidates say ca...,NBC News,https://web.archive.org/web/20190401012911/htt...
3,Break 'em up: Why Democrats running for presid...,"Break 'em up, say Dems: Why the 2020 field is ...",NBC News,https://web.archive.org/web/20190401012911/htt...
4,Trump calls Mueller probe an 'illegal takedown...,Justice Department's findings a 'complete and ...,NBC News,https://web.archive.org/web/20190401012911/htt...


In [87]:
nytimes = nytimes_df.drop(['author'],axis=1)
nytimes = nytimes.rename(index=str, columns={'link':'article_link'})
nytimes.head()

,title,summary,source,article_link
0,Joe Biden Scrambles to Stem Crisis After Lucy ...,The former vice presidents attempt to rebut h...,NYTimes,/web/20190401015034/https://www.nytimes.com/20...
1,"Who Is Lucy Flores, the Woman Accusing Joe Bid...","Ms. Flores, 39, is a former Nevada state assem...",NYTimes,/web/20190401015034/https://www.nytimes.com/20...
2,Joe Biden Says He Did Not Act Inappropriately ...,"The former vice president, accused by Ms. Flor...",NYTimes,/web/20190401015034/https://www.nytimes.com/20...
3,Elizabeth Warren Loses Finance Director as She...,"Money is a problem for the Democratic senator,...",NYTimes,/web/20190401015034/https://www.nytimes.com/20...
4,7 Takeaways From New Yorks $175 Billion Budget,The budget takes up thousands of pages and bil...,NYTimes,/web/20190401015034/https://www.nytimes.com/20...


In [90]:
allsides = allsides_df.drop(['news_bias'],axis=1)
allsides = allsides.rename(index=str, columns={'news_title':'title','news_summary':'summary','news_source':'source','news_lin':'article_link'})
allsides.head()

,title,summary,source,article_link
0,Democrats will meet in the next few weeks to d...,House Democrats will hold a meeting to discuss...,Washington Post,https://www.allsides.com/news/2019-04-21-0834/...
1,Why It Matters That Trump and Michael Cohen Ha...,"Fixer. The title wasn’t a formal one, but Mich...",New York Times - News,https://www.allsides.com/news/2019-04-21-0832/...
2,Confused about who’s who in the Mueller report...,"The Mueller report will be released Thursday, ...",Vox,https://www.allsides.com/news/2019-04-18-0720/...
3,Five key things to look for in the Mueller report,Barr has said the report has two parts: one on...,The Guardian,https://www.allsides.com/news/2019-04-18-0708/...
4,George Conway Reveals The Key Phrase To Watch ...,Attorney General William Barr has said he will...,HuffPost,https://www.allsides.com/news/2019-04-15-0925/...


In [30]:
news_df = allsides.append([breitbart,foxnews,nbc,nytimes],sort=False)
news_df.head()

#news_df.to_csv('news_df.csv',index=False)

NameError: name 'allsides' is not defined

## SVD on web scrapped news df

In [320]:
# import news_df data
news_df = pd.read_csv('news_df.csv')
len(news_df)

18495

In [321]:
news_df = news_df[pd.notnull(news_df['summary'])]
len(news_df)

16270

In [55]:
summaries = news_df['summary'].tolist()

In [353]:
# split news df into train and test set
from sklearn.model_selection import train_test_split

train_x, test_x = train_test_split(summaries,test_size=0.33,random_state=42)
test_x[:10]

["Yes, the government shutdown is to blame. No, it's not that big of a deal.",
 'Trump signs orders to speed up energy projectsThe president is trying to make it harder for states to block pipelines and other energy projects due to environmental concerns.',
 'Citizen journalist and pundit Tommy Robinson has announced his candidature for the European Parliament elections.',
 'Virginia Democratic Gov. Ralph Northam apologized Friday for appearing in a “racist and offensive” photo on his 1984 medical school yearbook page that...\r\n   ',
 'The man who endorsed 87 “national security” candidates and donated close to $500,000 during the 2014 midterms is back in action. Former United Nations...\r\n   ',
 'The 37-year-old mayor of South Bend, Ind., little known just a few months ago, has raised his political profile and online following in recent weeks.',
 "Three in four Americans who tuned in to President Trump's State of the Union address tonight approved of the speech he gave. Just a quarte

In [362]:
## clean_training data
clean_data(train_x)
train_x[:10]


['this week marks the anniversary of the battles of lexington and concord two hundred and fortyfour years ago farmers tradesmen laborers and mariners americans of all stripes came to together to defend themselves against the most professional army in the world',
 'former president barack obama issued a statement reacting to president donald trump s executive order to restrict immigration from highrisk countries in     ',
 'the decision by sen rand paul rky to vote in favor of blocking president trump s national emergency declaration for the border sets two things in motion on capitol hill',
 'tom fitton president of judicial watch said robert mueller s openness to obstruction of justice accusations against president donald trump amounted to his  final act of abuse of power  offering his remarks in a friday interview on siriusxm s breitbart news daily with host alex marlow',
 'president trump s presumptive nominee to the fed has written numerous opinion pieces questioning the role of wo

In [219]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(tokenizer = LemmaTokenizer())
#vectorizer = CountVectorizer(stop_words='english')
vectors_count = vectorizer.fit_transform(train_x).todense()
vectors.shape

(10900, 11884)

In [220]:
vocab = np.array(vectorizer.get_feature_names())
vocab[:20]

array(['aaron', 'aback', 'abandon', 'abbas', 'abbott', 'abby', 'abcs',
       'abdel', 'abdicate', 'abdication', 'abduct', 'abdul', 'abdulaziz',
       'abedin', 'abernathy', 'abeywardena', 'abhorrent', 'abide',
       'abigail', 'abilities'], dtype='<U36')

In [145]:
%time U,s,Vh = linalg.svd(vectors, full_matrices=False)

Wall time: 13min 36s


In [146]:
# save the matrices
np.savez('model_nopunc.npz',U=U,s=s,Vh=Vh)

In [147]:
num_top_words = 8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [149]:
show_topics(Vh[:20])

['trump president said donald house mueller report white',
 'president vice biden donald joe trump border obama',
 'trump mueller report donald special campaign robert counsel',
 'president biden report presidential vice democratic special joe',
 'president house mueller white special counsel report robert',
 'said attorney general barr counsel william special committee',
 'presidential democratic new campaign candidate york state clinton',
 'new york state rep times jersey trumps report',
 'mueller report said white biden vice joe reportthe',
 'democrats says rep mueller report republican border attempted',
 'rep donald counsel special robert news fox democratic',
 'rep donald biden democrats new joe committee vice',
 'democrats counsel special says states democratic does administration',
 'democratic presidential attorney mueller general report candidates president',
 'news fox border security administration told biden national',
 'democrats news fox campaign white buttigieg said san

### SVD w/ TFIDF

In [224]:
#vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectorizer_tfidf = TfidfVectorizer(tokenizer=LemmaTokenizer())
vectors = vectorizer_tfidf.fit_transform(train_x).todense()
vectors.shape

(10900, 11406)

In [225]:
vocab_tfidf = np.array(vectorizer_tfidf.get_feature_names())
vocab_tfidf[:10]

array(['aaron', 'aback', 'abandon', 'abbas', 'abbott', 'abby', 'abcs',
       'abdel', 'abdicate', 'abdication'], dtype='<U36')

In [230]:
%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

Wall time: 9min 40s


In [156]:
np.savez('model_tfidf.npz',U=U,s=s,Vh=Vh)

In [231]:
U.shape, s.shape, Vh.shape

((10900, 10900), (10900,), (10900, 11406))

In [ ]:
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [233]:
show_topics(Vh[:15])

['trump president mueller say report house former counsel',
 'mueller report counsel special release reportthe redactions text',
 'political read stories unit matter important brief meet',
 'house attorney general white barr counsel political committee',
 'attorney general special barr counsel campaign presidential find',
 'trump president donald mueller border text redactions take',
 'say state would bill general clinton hillary attorney',
 'attorney general barr former william biden vice testimony',
 'campaign say presidential investigation trump robert russia democrats',
 'clinton hillary campaign former email arrest wikileaks assange',
 'say biden treasury former vice secretary democrats request',
 'clinton hillary democrats house state pelosi speaker nancy',
 'clinton investigation democrats robert hillary obama border call',
 'say white campaign officials find barr exoneration total',
 'wikileaks assange founder biden call arrest government charge']

### Randomised SVD

In [226]:
%time u, s, v = decomposition.randomized_svd(vectors,15)

Wall time: 3.29 s


In [229]:
np.savez('model_RandSVD.npz',U=u,s=s,Vh=v)

In [227]:
u.shape, s.shape, v.shape

((10900, 15), (15,), (15, 11406))

In [228]:
show_topics(v)

['trump president mueller say report house former counsel',
 'mueller report counsel special release reportthe redactions text',
 'political read stories unit matter important brief meet',
 'presidential mueller biden text redactions democratic release vice',
 'house white reportthe redactions text mueller security subpoena',
 'presidential democratic former house biden general attorney barr',
 'former president vice biden counsel special white robert',
 'attorney general barr william former biden vice president',
 'state president unite barr former justice department exoneration',
 'clinton hillary campaign former email arrest wikileaks secretary',
 'say biden treasury former vice secretary request mnuchin',
 'security border homeland administration department officials national nielsen',
 'committee campaign trump pick donors inaugural qualifiedtrumps marquette',
 'committee robert investigation former senate donald bill court',
 'buttigieg white say pete mayor donald presidential co

### LDA

In [301]:
vectorizer_lda = CountVectorizer(tokenizer=LemmaTokenizer(),max_df=0.5,min_df=10)
vectors_lda = vectorizer_lda.fit_transform(train_x).todense()
vectors_lda.shape

(10900, 2616)

In [306]:
vectorizer_lda_tfidf = TfidfVectorizer(tokenizer=LemmaTokenizer(),max_df=0.5,min_df=10)
vectors_lda_tfidf = vectorizer_lda_tfidf.fit_transform(train_x).todense()
vectors_lda_tfidf.shape

(10900, 2616)

In [307]:
vocab_lda_tfidf = np.array(vectorizer_lda_tfidf.get_feature_names())
vocab_lda_tfidf[:10]

array(['abandon', 'ability', 'able', 'abolish', 'abortion', 'abortions',
       'abrams', 'abroad', 'abrupt', 'absent'], dtype='<U19')

In [302]:
vocab_lda = np.array(vectorizer_lda.get_feature_names())
vocab_lda[:10]

array(['abandon', 'ability', 'able', 'abolish', 'abortion', 'abortions',
       'abrams', 'abroad', 'abrupt', 'absent'], dtype='<U19')

In [308]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(15,random_state=0,max_iter=20)
ldatransfrom = lda.fit(vectors_lda_tfidf)


In [309]:
H1 = ldatransfrom.components_

In [275]:
for j,t in enumerate(H1):
    print(j,t)

0 [0.06666671 0.06666667 0.06666671 ... 0.06666667 0.06666667 0.06666667]
1 [0.06666667 0.06666667 0.06666674 ... 0.06666672 0.06666667 1.06666667]
2 [0.06666667 0.06666677 3.57932723 ... 0.06666667 0.06666669 0.06666667]
3 [0.06666674 0.06666667 0.06666689 ... 0.0666667  0.06666667 0.06666667]
4 [0.06666669 0.06666667 0.0666668  ... 1.06666645 0.06666667 0.06666667]
5 [0.06666667 0.06666667 0.0666667  ... 0.06666667 0.06666667 0.06666667]
6 [0.06666667 0.06666667 0.06666669 ... 0.06666671 0.06666667 0.06666667]
7 [0.06666669 0.06666667 0.06666669 ... 0.06666667 0.06666667 0.06666667]
8 [0.0666667  3.06666656 6.27264475 ... 0.06666668 0.06666667 0.06666667]
9 [3.06666627 0.06666667 0.06666683 ... 0.06666667 0.06666667 0.06666667]
10 [0.06666667 0.06666667 2.29439829 ... 0.0666668  0.06666667 0.06666667]
11 [0.0666668  0.06666667 1.12029557 ... 0.06666667 0.06666667 0.06666667]
12 [0.06666667 0.06666667 0.06666681 ... 0.06666672 1.06666664 0.06666667]
13 [0.06666673 0.06666667 2.0666664

In [369]:
num_top_words = 8

def show_topics(a):
    for j, t in enumerate(a):
        top_words = lambda w: [vocab_lda[i] for i in np.argsort(w)[:-num_top_words-1:-1]]
        topic_words = ([top_words(t)])
        msg = 'topic %i ' % (j+1)
        word_list = [' '.join(words) for words in topic_words]
        msg+= ' '.join(word_list)
        print(msg)

In [310]:
show_topics(H1)

topic 1 mueller redactions text thursday report release read reportthe
topic 2 former campaign presidential president trump democratic candidate chris
topic 3 political meet important days matter stories brief press
topic 4 state party politics say president trump senate court
topic 5 president trump would conway state say percent return
topic 6 counsel mueller special report attorney robert trump general
topic 7 president obama state barack trump bush former donald
topic 8 trump president border michael cohen wall house national
topic 9 trump president ilhan omar hill twitter deal remark
topic 10 president trump say israel people black elect women
topic 11 trump president security say secretary former policy treasury
topic 12 pelosi speaker house trump nancy president impeachment democrats
topic 13 buttigieg pete presidential say president view trump south
topic 14 clinton hillary email former president trump news clintons
topic 15 grow trump secretary president threaten say julian ha

In [311]:
np.savez('model_lda.npz',H1 = H1)

In [318]:

prepared_news_story = pyLDAvis.sklearn.prepare(lda,vectors_lda_tfidf,vectorizer_lda_tfidf,sort_topics=False)

In [319]:
pyLDAvis.display(prepared_news_story)

## Topics by news sources

In [383]:
news_df = pd.read_csv('news_df.csv')

In [384]:
fox_filter = news_df[news_df['source'].str.contains('Fox.*')]

len(fox_filter)

5682

In [385]:
fox_filter.head()

,title,summary,source,article_link
13,Trump World ramps up campaign to turn tables i...,As Democratic leaders tentatively took impeach...,Fox News,https://www.allsides.com/news/2019-04-24-0813/...
28,‘Shadowy’ dark money network behind left-wing ...,An expansive network of “shadowy” dark money d...,Fox News,https://www.allsides.com/news/2019-04-17-0816/...
40,"Students, alumni outraged, 'shaking' after Vic...",Vice President Mike Pence is getting pushback ...,Fox News,https://www.allsides.com/news/2019-04-15-0934/...
44,House Democrats increase demands for Trump tax...,House Democrats on Saturday increased their de...,Fox News,https://www.allsides.com/news/2019-04-13-1355/...
55,"Comey scoffs at Barr testimony, claims ‘survei...",Former FBI Director James Comey joined the cho...,Fox News,https://www.allsides.com/news/2019-04-12-0750/...


In [386]:
fox_summaries = fox_filter[pd.notnull(fox_filter['summary'])]['summary'].to_list()
len(fox_summaries)

3473

In [387]:
fox_summaries[:10]

['As Democratic leaders tentatively took impeachment proceedings off the docket this week, the White House put payback on the front burner -- calling for...\r\n   ',
 'An expansive network of “shadowy” dark money donors has grown to rival the influence of the conservative Koch brothers -- pumping millions into left-wing...\r\n   ',
 'Vice President Mike Pence is getting pushback from Taylor University students and alumni after the small evangelical Christian school tapped the former...\r\n   ',
 'House Democrats on Saturday increased their demands for the IRS to give them access to President Trump’s tax returns -- foreshadowing a lengthy legal battle...\r\n   ',
 'Former FBI Director James Comey joined the chorus of Democratic critics complaining about Attorney General Bill Barr’s testimony this week that “spying did...\r\n   ',
 'Hours before Alexandria Ocasio-Cortez called for an “agenda of reparations” at an Al Sharpton-sponsored event in New York on Friday, she was slammed as “...\

In [388]:
clean_data(fox_summaries)

['as democratic leaders tentatively took impeachment proceedings off the docket this week the white house put payback on the front burner  calling for     ',
 'an expansive network of  shadowy  dark money donors has grown to rival the influence of the conservative koch brothers  pumping millions into leftwing     ',
 'vice president mike pence is getting pushback from taylor university students and alumni after the small evangelical christian school tapped the former     ',
 'house democrats on saturday increased their demands for the irs to give them access to president trump s tax returns  foreshadowing a lengthy legal battle     ',
 'former fbi director james comey joined the chorus of democratic critics complaining about attorney general bill barr s testimony this week that  spying did     ',
 'hours before alexandria ocasiocortez called for an  agenda of reparations  at an al sharptonsponsored event in new york on friday she was slammed as       ',
 'msnbc s town hall featuring re

In [401]:
#LDA model
fox_vectorizer_tfidf = TfidfVectorizer(tokenizer=LemmaTokenizer(),max_df=0.5,min_df=10) #TFIDF vectorizer
#fox_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),max_df=0.5,min_df=10) #Countvectorizer
fox_vectors = fox_vectorizer_tfidf.fit_transform(fox_summaries).todense()  # transfrom using tfidf vectorizer
#fox_vectors = fox_vectorizer.fit_transform(fox_summaries).todense() #transform using CountVectorizer
fox_vectors.shape

(3473, 1082)

In [402]:
fox_vocab = np.array(fox_vectorizer_tfidf.get_feature_names())
fox_vocab[:10]

array(['abruptly', 'abuse', 'accept', 'access', 'accord', 'accusations',
       'accuse', 'acknowledge', 'across', 'act'], dtype='<U15')

In [396]:
fox_vocab_count = np.array(fox_vectorizer.get_feature_names())
fox_vocab_count[:10]

array(['abruptly', 'abuse', 'accept', 'access', 'accord', 'accusations',
       'accuse', 'acknowledge', 'across', 'act'], dtype='<U15')

In [403]:
foxlda = lda.fit(fox_vectors)

In [404]:
fH1 = foxlda.components_

In [405]:
num_top_words = 8

def show_topics_fox(a):
    for j, t in enumerate(a):
        top_words = lambda w: [fox_vocab[i] for i in np.argsort(w)[:-num_top_words-1:-1]]
        topic_words = ([top_words(t)])
        msg = 'topic %i ' % (j+1)
        word_list = [' '.join(words) for words in topic_words]
        msg+= ' '.join(word_list)
        print(msg)

In [406]:
show_topics_fox(fH1)

topic 1 trump president white democrats report impeachment corke house
topic 2 ilhan omar comment candidate presidential ocasiocortez democratic alexandria
topic 3 clinton hillary executive primary college free order democratic
topic 4 border emergency southern trump friends national declaration say
topic 5 deal speaker green care health pelosi nancy push
topic 6 special reaction analysis report barr mueller counsel general
topic 7 trump michael president cohen host radio senior political
topic 8 house committee trump president chairman judiciary white say
topic 9 beto rourke campaign ocasiocortez story file poll independent
topic 10 trump mueller president former attorney counsel probe democrats
topic 11 security homeland secretary nielsen trump border kirstjen act
topic 12 former president vice biden obama state women trump
topic 13 rally say manafort paul years florida city trump
topic 14 democratic sanders candidates bernie field presidential strategist doocy
topic 15 press return 

In [407]:
np.savez('fox_topic_lda.npz',foxlda)

In [408]:
warnings.filterwarnings('ignore')
prepared_foxnews_story = pyLDAvis.sklearn.prepare(lda,fox_vectors,fox_vectorizer_tfidf,sort_topics=False)

In [409]:
pyLDAvis.display(prepared_foxnews_story)

In [411]:
nytimes_filter = news_df[news_df['source'].str.contains('NYTimes|New York Times')]
len(nytimes_filter)

3075

In [412]:
nytimes_filter.head()

,title,summary,source,article_link
1,Why It Matters That Trump and Michael Cohen Ha...,"Fixer. The title wasn’t a formal one, but Mich...",New York Times - News,https://www.allsides.com/news/2019-04-21-0832/...
19,House Chairman Says Treasury’s Reasons to With...,A Democratic House chairman on Saturday castig...,New York Times - News,https://www.allsides.com/news/2019-04-13-1236/...
78,Trump and Republicans Seek to Turn the Tables ...,President Trump and his Republican allies went...,New York Times - News,https://www.allsides.com/news/2019-03-26-0750/...
93,"After Week of Infighting, Democrats Wonder Whe...",Left unanswered is a question that transcends ...,New York Times - News,https://www.allsides.com/news/2019-03-10-0806/...
138,"Manafort’s Lawyers, Saying He Learned ‘Harsh L...","Lawyers for Paul Manafort, President Trump’s f...",New York Times - News,https://www.allsides.com/news/2019-03-02-0736/...


In [416]:
nytimes_summaries = nytimes_filter[pd.notnull(nytimes_filter['summary'])]['summary'].tolist()
nytimes_summaries[:10]

['Fixer. The title wasn’t a formal one, but Michael D. Cohen wholeheartedly embraced the role as a lawyer at the Trump Organization. It amounted to serving as...\r\n   ',
 'A Democratic House chairman on Saturday castigated the Treasury Department for failing to meet his deadline to furnish President Trump’s tax returns,...\r\n   ',
 'President Trump and his Republican allies went on the offensive on Monday, vowing to pursue and even punish those responsible for the Russia investigation...\r\n   ',
 'Left unanswered is a question that transcends partisan politics: In an era of provocation, how should Congress respond when its members say hateful or...\r\n   ',
 'Lawyers for Paul Manafort, President Trump’s former campaign chairman, asked a federal judge in Northern Virginia on Friday to show leniency when he...\r\n   ',
 'President Trump ordered his chief of staff to grant his son-in-law and senior adviser, Jared Kushner, a top-secret security clearance last year, overruling...\r\n   '

In [417]:
clean_data(nytimes_summaries)

['fixer the title wasn t a formal one but michael d cohen wholeheartedly embraced the role as a lawyer at the trump organization it amounted to serving as     ',
 'a democratic house chairman on saturday castigated the treasury department for failing to meet his deadline to furnish president trump s tax returns     ',
 'president trump and his republican allies went on the offensive on monday vowing to pursue and even punish those responsible for the russia investigation     ',
 'left unanswered is a question that transcends partisan politics in an era of provocation how should congress respond when its members say hateful or     ',
 'lawyers for paul manafort president trump s former campaign chairman asked a federal judge in northern virginia on friday to show leniency when he     ',
 'president trump ordered his chief of staff to grant his soninlaw and senior adviser jared kushner a topsecret security clearance last year overruling     ',
 'michael d cohen donald j trump s former la

In [418]:
nytimes_vectorizer_tfidf = TfidfVectorizer(tokenizer=LemmaTokenizer(),max_df=0.5,min_df=10)
nytimes_vectors = nytimes_vectorizer_tfidf.fit_transform(nytimes_summaries).todense()
nytimes_vectors.shape

(3075, 1100)

In [419]:
nytimes_vocab = np.array(nytimes_vectorizer_tfidf.get_feature_names())
nytimes_vocab[:10]

array(['ability', 'abortion', 'abrupt', 'absent', 'access', 'accord',
       'account', 'accusations', 'accuse', 'across'], dtype='<U17')

In [420]:
nytimes_lda = lda.fit(nytimes_vectors) #lda

In [421]:
nyH1 = nytimes_lda.components_

In [424]:
num_top_words = 8

def show_topics_nytimes(a):
    for i,t in enumerate(a):
        top_words = lambda w:[nytimes_vocab[i] for i in np.argsort(w)[:-num_top_words-1:-1]]
        topic_words = ([top_words(t)])
        msg = 'topic %i ' % (j+1)
        word_list = [' '.join(word) for word in topic_words]
        msg+= ' '.join(word_list)
        print(msg)
        

In [425]:
show_topics_nytimes(nyH1)

topic 15 president security time counsel trump hill say special
topic 15 attorney general hillary clinton report president special counsel
topic 15 say attorney general trump barr president william congress
topic 15 promise clinton champion wikileaks home continue describe hear
topic 15 democratic presidential candidates senator sanders speech public economic
topic 15 politics counsel trump president busy stories miss week
topic 15 state change mayor former agency president inquiry members
topic 15 political administration president house people previously state officials
topic 15 care base would republicans provide house bill president
topic 15 president trump women report mueller team find potential
topic 15 federal buttigieg president return criminal campaign live record
topic 15 seem tuesday assault president sexual draw comment respond
topic 15 president treasury vice state say unite notably former
topic 15 answer trump bush president seek find critics could
topic 15 trump preside

In [426]:
np.savez('nytimes_topics_lda.npz',nytimes_lda)

In [427]:
warnings.filterwarnings('ignore')
prepared_nytimes_story = pyLDAvis.sklearn.prepare(lda,nytimes_vectors,nytimes_vectorizer_tfidf,sort_topics=False)

In [428]:
pyLDAvis.display(prepared_nytimes_story)

In [2]:
news_df = pd.read_csv('news_df.csv')
breitbart_filter = news_df[news_df['source'].str.contains('Breitbart.*')]
len(breitbart_filter)

3072

In [4]:
breitbart_summaries = breitbart_filter[pd.notnull(breitbart_filter['summary'])]['summary'].tolist()
len(breitbart_summaries)

3066

In [10]:
clean_data(breitbart_summaries)

['the notion that payments to his personal lawyer might implicate president donald trump in wrongdoing was dealt a blow wednesday by drug giant novartis      ',
 'during a portion of an interview with nbc news national political reporter heidi przybyla former senate majority leader harry reid dnv stated  the less     ',
 'sen ted cruz rtx recommended usingantitrust laws to curb the  massive power  of facebook and other technology companies during a tuesday interview with     ',
 'tuesday at the pennsylvania conference for women former first lady michelle obama said when she attended the state of the union address she would notice     ',
 'during an interview on siriusxm progress set to air on tuesday former democratic presidential nominee hillary clinton stated it s  deeply troubling  that     ',
 'former deputy adviser to the president dr sebastian gorka discussed his exit from the white house with siriusxm host alex marlow on thursday s breitbart     ',
 'msnbc host chris matthews sa

In [13]:
breitbart_vectorizer_tfidf = TfidfVectorizer(tokenizer=LemmaTokenizer(),max_df=0.5,min_df=10)
breitbart_vectors = breitbart_vectorizer_tfidf.fit_transform(breitbart_summaries).todense()
breitbart_vectors.shape

(3066, 1095)

In [17]:
breitbart_vocab = np.array(breitbart_vectorizer_tfidf.get_feature_names())
breitbart_vocab[:10]

array(['abortion', 'abortions', 'abrams', 'abuse', 'accept', 'accord',
       'account', 'accusations', 'accuse', 'across'], dtype='<U15')

In [15]:
lda = LatentDirichletAllocation(15,random_state=0,max_iter=20)
breit_lda = lda.fit(breitbart_vectors)

In [16]:
bH1 = breit_lda.components_

In [25]:
num_top_words = 8

def show_topics_breitbart(a):
    for j,t in enumerate(a):
        top_words = lambda w: [breitbart_vocab[i] for i in np.argsort(w)[:-num_top_words-1:-1]]
        topic_words = ([top_words(t)])
        msg = 'topic %i ' % (j+1)
        word_list = [' '.join(word) for word in topic_words]
        msg+= ' '.join(word_list)
        print(msg)

In [26]:
show_topics_breitbart(bH1)

topic 1 first president trump white lady school house speak
topic 2 illegal state unite alien foreign immigration federal american
topic 3 buttigieg pete presidential mayor candidate sanders hopeful bernie
topic 4 york time publish antisemitic medicare cartoon jewish israel
topic 5 president former presidential news trump obama sanders biden
topic 6 european party brexit parliament elections poll nigel farage
topic 7 mueller counsel special robert report attorney barr general
topic 8 trump president say clinton state donald hillary presidential
topic 9 city pompeo ohio public last story southern expect
topic 10 twitter shoot woman parenthood prolife paris black write
topic 11 secretary court former security homeland nielsen president supreme
topic 12 biden former vice president touch chicago women actor
topic 13 border trump president migrants immigration state unite donald
topic 14 minister prime theresa party netanyahu aviv benjamin government
topic 15 house trump say president pelos

In [28]:
warnings.filterwarnings('ignore')
prepared_breitbart_story = pyLDAvis.sklearn.prepare(lda,breitbart_vectors,breitbart_vectorizer_tfidf,sort_topics = False)

In [29]:
pyLDAvis.display(prepared_breitbart_story)

In [30]:
nbc_filter = news_df[news_df['source'].str.contains('NBC.*')]
len(nbc_filter)

3098

In [31]:
nbc_summaries = nbc_filter[pd.notnull(nbc_filter['summary'])]['summary'].tolist()
len(nbc_summaries)

3098

In [32]:
clean_data(nbc_summaries)

[' if the drugs don t stop or largely stop we re going to put tariffs on mexico and products  and if that doesn t stop the drugs we close the border      ',
 'el paso texas   edgar salazar a dental surgeon in ciudad juarez across the border in mexico stood a few feet from slowmoving traffic on a bridge     ',
 'roger stone a former adviser to president donald trump was indicted by a federal grand jury as part of special counsel robert muellers probe and arrested     ',
 'nick sandmann one of the high school students videotaped last week in an apparent confrontation with a native american activist at the lincoln memorial in     ',
 'officials said that the     squaremile park will close thursday morning temporarily but did not say when they expected to reopen it to visitors',
 'bush s toughoncrime stance in his      campaign has been cited by historians as helping to drive decades of politics low on justice or equity',
 'the minority party has to start playing the long game and playing 

In [33]:
nbc_vectorizer_tfidf = TfidfVectorizer(tokenizer = LemmaTokenizer(),max_df=0.5,min_df=10)
nbc_vectors = nbc_vectorizer_tfidf.fit_transform(nbc_summaries).todense()
nbc_vectors.shape

(3098, 1380)

In [34]:
nbc_vocab = np.array(nbc_vectorizer_tfidf.get_feature_names())
nbc_vocab[:10]

array(['ability', 'able', 'abolish', 'abortion', 'aboutthe', 'abroad',
       'absolutely', 'abuse', 'access', 'accord'], dtype='<U21')

In [35]:
nbc_lda = lda.fit(nbc_vectors)

In [36]:
nH1 = nbc_lda.components_

In [37]:
def show_topics_nbc(a):
    for j,t in enumerate(a):
        top_words = lambda w: [nbc_vocab[i] for i in np.argsort(w)[:-num_top_words-1:-1]]
        topic_words = ([top_words(t)])
        msg = 'topic %i ' % (j+1)
        word_list = [' '.join(word) for word in topic_words]
        msg+= ' '.join(word_list)
        print(msg)

In [38]:
show_topics_nbc(nH1)

topic 1 mueller redactions text thursday release reportthe read report
topic 2 political stories press unit brief matter important days
topic 3 president action security think leave mueller embolden bound
topic 4 buttigieg pete paso hard grow people college back
topic 5 energy make didnt defend deputy say whether mueller
topic 6 say spy office hold pelosi close border tell
topic 7 subpoena report house say chairman mueller north nadler
topic 8 treasury israel say issue center serious mnuchin lesbian
topic 9 house white arrest say worry separation nominees concern
topic 10 attempt coup lightfoot pence judge rally democrats mayor
topic 11 counsel special find conclude exoneration exonerate crime departments
topic 12 profit trillion first would enjoy warren presidential plan
topic 13 former biden democratic president poll race campaign presidential
topic 14 question citizenship census state florida would court report
topic 15 deal bill population administration play proceed join percent


In [39]:
warnings.filterwarnings('ignore')
prepared_nbc_story = pyLDAvis.sklearn.prepare(lda,nbc_vectors,nbc_vectorizer_tfidf,sort_topics=False)

In [40]:
pyLDAvis.display(prepared_nbc_story)